In [7]:
import mlflow.pyfunc
import pandas as pd
import mbr_kernel as mkn
from sklearn.model_selection import train_test_split

NOM_MODELE_ML='XGBoost_NoExt'
VERSION_MODEL = 1
POSSIBLE_STAGES = ['Staging', 'Production']

TEST_SIZE = 0.2
RANDOM_STATE = 42

In [8]:
df_train = pd.read_csv('./input_data/application_train.csv')
df = mkn.full_feature_engineering(df_input=df_train, df_folder='./input_data/',encoding_treshold=0.04, nan_treshold=0.4)
X = df.drop(columns = ['SK_ID_CURR', 'TARGET'])
y = df['TARGET']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y)
print(f"X_train : {X_train.shape} // X_test : {X_test.shape}")
print(f"y_train : {y_train.shape} // y_test : {y_test.shape}")

Forme initiale du Dataframe :  (307511, 122)


/home/maximorose/git_repositories/cln_oc_p07/outils_feature_engineering_810.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_df.drop(columns=['taux_nan'], inplace=True)
/home/maximorose/git_repositories/cln_oc_p07/mbr_kernel.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
/home/maximorose/git_repositories/cln_oc_p07/mbr_kernel.py:186: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

Forme du Dataframe apres traitement :  (219656, 122)

Application dataframe shape:  (219656, 14)
application processing - done in 1s
Forme initiale du Dataframe :  (1716428, 17)
Forme du Dataframe apres traitement :  (1716428, 17)
Bureau dataframe shape:  (305811, 8)
Bureau and bureau_balance processing - done in 7s
Forme initiale du Dataframe :  (1670214, 37)


/home/maximorose/git_repositories/cln_oc_p07/outils_feature_engineering_810.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_df.drop(columns=['taux_nan'], inplace=True)


Forme du Dataframe apres traitement :  (1234578, 37)
Forme initiale du Dataframe :  (13605401, 8)
Forme du Dataframe apres traitement :  (13605401, 8)
Previous Application dataframe shape:  (338308, 12)
Previous application processing - done in 23s
Pos-Cash dataframe shape:  (337252, 3)
Pos-Cash processing - done in 5s
Installement dataframe shape:  (339587, 7)
Installment processing - done in 13s
CC dataframe shape:  (103558, 6)
CC processing - done in 6s
X_train : (175724, 48) // X_test : (43932, 48)
y_train : (175724,) // y_test : (43932,)


In [13]:
import numpy as np

print("Taille globale y_test : ", y_test.shape[0])
print("Nombre target = 1 : ", y_test.sum())
print("Part target = 1 (%) : ", np.round((y_test.sum()/y_test.shape[0])*100, 2))

Taille globale y_test :  43932
Nombre target = 1 :  3262.0
Part target = 1 (%) :  7.43


In [14]:
0.37*3262

1206.94

In [15]:
(43932-3262)*0.3

12201.0

# Transitioning model stages

cf [ici](https://mlflow.org/docs/latest/model-registry.html#transitioning-an-mlflow-models-stage)

In [9]:
from mlflow import MlflowClient

In [10]:
client = MlflowClient()
client.transition_model_version_stage(
    name=NOM_MODELE_ML, version=VERSION_MODEL, stage=POSSIBLE_STAGES[1]
)

<ModelVersion: aliases=[], creation_timestamp=1686815585758, current_stage='Production', description='', last_updated_timestamp=1686836450777, name='XGBoost_NoExt', run_id='86ac7fc843804367a1ff5e938a0c2149', run_link='', source='file:///home/maximorose/git_repositories/cln_oc_p07/mlruns/0/86ac7fc843804367a1ff5e938a0c2149/artifacts/XGBoost', status='READY', status_message=None, tags={}, user_id=None, version=1>

# Fetching model

Source : [ici](https://mlflow.org/docs/latest/model-registry.html#id8)

## 1. by version ref'

In [5]:
model_name = NOM_MODELE_ML
model_version = VERSION_MODEL

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

## 2. by stage

In [6]:
model_name = NOM_MODELE_ML
stage =POSSIBLE_STAGES[0]

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{stage}")

# Serving model

__N.B :__ Necessite le demarrage preablable de MLFlow

From Terminal :
- Set environment variable for the tracking URL where the Model Registry resides
- pyenv for version management related to flavors
- Call serving function and specify port to avoid exception


`export MLFLOW_TRACKING_URI=http://localhost:5000`

`export PATH="$HOME/.pyenv/bin:$PATH"`

`mlflow models serve -m "models:/Best_XGBoost_EXTvar_mkn/Staging" -p 5001`


Apres quoi, le terminal affichera l'URI pour appeler le model :
`http://127.0.0.1:5001`
